In [1]:
%pylab inline
import numpy as np
import fastcat as fc
#import treecorr as tc
import healpy as hp
#import os
import sys
import glob, os
from scipy import integrate
from scipy.special import erf
#sys.path.append("..")


  
#photozs that we have
#pzs=[]
#pzs.append(("Gauss",fc.photoz.PhotoZGauss(0.02)))
#pzs.append(("TwoPop",fc.photoz.PhotoZTwoPop(0.02,0.75,3.)))
#pzs.append(("HiddenVar",fc.photoz.PhotoZHiddenVar(0.02,[0.1,0.2],[0.6,0.65])))
#pzs.append(("Franzona",fc.photoz.PhotoZHist("/home/nil/Downloads/LSSWG/Franzona/pzdist.txt")))    
  
os.environ['DESC_LSS_ROOT']='/home/nil/Downloads/LSSWG/'  

#import the pixel edges

dec_bin_edges=np.loadtxt("/home/nil/LNmocks/Gauss_distribute/decbin_edges_20-60.txt")
ra_bin_edges=np.loadtxt("/home/nil/LNmocks/Gauss_distribute/rabin_edges_100.txt")
z_bin_edges=np.loadtxt("/home/nil/LNmocks/Gauss_distribute/zbin_edges.txt")  



Populating the interactive namespace from numpy and matplotlib


In [2]:
dir="/home/nil/LNmocks/160620+GaussPZ_0.01+FullSky+realspace+subsamp_1000000/"

os.chdir(dir)
for file in glob.glob("*.h5"):
    file2 = open("/home/nil/LNmocks/Gauss_distribute/vox_catalogs/"+ file + ".dat", "w")
    data=fc.Catalog(read_from=dir + file)
    pz=data.photoz #pz stores the type of photoz applied to the catalog
    phoz=pz.getMeanRMS(data)[0]
    sigz=pz.getMeanRMS(data)[1]
    ddata=np.c_[data['ra'],data['dec'],phoz,sigz]
    
    ddata=ddata[ddata[:,0] >= 0.0]
    ddata=ddata[ddata[:,0] <= 100.0]
    ddata=ddata[ddata[:,1] >= 20.0]
    ddata=ddata[ddata[:,1] <= 60.0]
    ddata=ddata[ddata[:,2] >= 0.5]
    ddata=ddata[ddata[:,2] <= 0.9]
    print len(ddata), len(data['ra'])
   
    ra_data=ddata[:,0]
    dec_data=ddata[:,1]
    z_data=ddata[:,2]
    sigma=ddata[:,3]  

    z_up=z_bin_edges[1:]        #upper z bounds, excluded the first bound
    z_lo=z_bin_edges[:len(z_bin_edges)-1]    #lower z bounds, excluded the last bound
    z_prob_pix=np.zeros((len(ra_data),len(z_bin_edges)-1))

    #print z_prob_pix[0], len(z_prob_pix[0])

    for i in range(len(ra_data)):
         z_prob_pix[i]=0.5*(erf((z_up - z_data[i])/(np.sqrt(2)*sigma[i])) - erf((z_lo - z_data[i])/(np.sqrt(2)*sigma[i])))

    #print z_prob_pix         

    #Replace the ra and dec of the galxies with the mid value of ra and dec pixel in which they are located
    for i in range(len(ra_data)):
         for j in range(len(ra_bin_edges)-1):
                if  ra_bin_edges[j] <= ra_data[i] < ra_bin_edges[j+1]:
                    ra_data[i]=(ra_bin_edges[j] + ra_bin_edges[j+1])*0.5 
                  
    for i in range(len(dec_data)):
         for j in range(len(dec_bin_edges)-1):
                if  dec_bin_edges[j]  <= dec_data[i] < dec_bin_edges[j+1]:   #CHECK THIS LINE FOR DEC!!!!
                    dec_data[i]=(dec_bin_edges[j] + dec_bin_edges[j+1])*0.5 
                
    galaxy_prob_pix=np.c_[ra_data,dec_data,z_prob_pix]  

    #print galaxy_prob_pix



    ###########add all the rows with the same ra and dec
    d = {}
    for sub in galaxy_prob_pix:
        d.setdefault(tuple(sub[:2]),[]).append(sub[2:])#
    
    gal_pix= np.array([k+tuple(map(sum, zip(*v))) for k,v in d.items()]) 

    #print len(gal_pix)

    #print len(gal_pix), len(gal_pix[0])

    z_mid=[(z_bin_edges[i]+z_bin_edges[i+1])/2 for i in range(len(z_bin_edges)-1)]
    #print len(z_mid)    
    gw=[]
   
    for i in range(len(gal_pix)):
        gw.append(np.c_[[gal_pix[:,:2][i]]*len(z_mid),z_mid])  #list of arrays (ra,dec,zmid)
        
    #print len(gw)        

    gr= np.reshape(np.ravel(gw),(len(gal_pix)*len(z_mid),3)) # reshape to make it into list of list

    #print len(gr)


    hh= np.delete(gal_pix,0,1)  #delete ra column

    gg= np.delete(hh,0,1)   # delete dec column

    prbs=np.ravel(np.reshape(np.ravel(gg),(len(gr[:,0]),1)))  #take the prob rows and turn them into one long vector to match the number of pixels

    #print len(prbs), len(gr[:,0])

    prob_data=np.c_[gr[:,0],gr[:,1],gr[:,2],prbs]

    ra=prob_data[:,0]
    dec=prob_data[:,1]
    rdsft=prob_data[:,2]
    w=prob_data[:,3]

    print len(ra)
            

    
    for index in range(len(ra)):
         file2.write(str(ra[index]) + "   " + str(dec[index])  + "   " + str(rdsft[index])  + "   " + str(w[index]) +  "\n")
    file2.close()

39175 1000000
334880
39230 1000000
334100
39240 1000000
335244
39429 1000000
335036
39161 1000000
334464
39001 1000000
333736
38947 1000000
334620
39056 1000000
334880
39265 1000000
334724
39581 1000000
334568
39315 1000000
334776
38739 1000000
334776
38543 1000000
334256
38699 1000000
334464
38596 1000000
333996
39369 1000000
334256
39351 1000000
334568
38821 1000000
334412
39007 1000000
334724
39316 1000000
334984
38905 1000000
334204
39215 1000000
334672
39403 1000000
334516
38650 1000000
334464
39404 1000000
333944
38753 1000000
333996
38844 1000000
334568
39051 1000000
334828
39023 1000000
334464
39027 1000000
334308
38945 1000000
334724
38947 1000000
334412
39147 1000000
334256
38828 1000000
334204
38717 1000000
334204
39186 1000000
334464
39311 1000000
334776
38463 1000000
333996
39040 1000000
334776
39277 1000000
334672
39216 1000000
334620
39062 1000000
334568
39345 1000000
334412
39657 1000000
334672
38689 1000000
334152
39380 1000000
335036
39357 1000000
334516
39170 1000000

In [ ]:
dir="/home/nil/LNmocks/160620+GaussPZ_0.01+FullSky+realspace+subsamp_1000000/"
os.chdir(dir)
for file in glob.glob("*.h5"):
    data=fc.Catalog(read_from=dir + file)
    pz=data.photoz #pz stores the type of photoz applied to the catalog
    phoz=pz.getMeanRMS(data)[0]
    sigz=pz.getMeanRMS(data)[1]
    data=data[-88.66 < data['dec']]   ####Only works for TruZ, Gauss and doubleGauss!
    data=data[data['dec']< 88.66]
    
    ra_data=data['ra']
    dec_data=data['dec']
    z_data=data['z']
    
    z_up=z_bin_edges[1:]        #upper z bounds, excluded the first bound
    z_lo=z_bin_edges[:len(z_bin_edges)-1]    #lower z bounds, excluded the last bound
    z_prob_pix=np.zeros((len(ra_data),len(z_bin_edges)-1))

    for i in range(len(ra_data)):
        z_prob_pix[i]=0.5*(erf((z_up - z_data[i])/(np.sqrt(2)*sigz[i])) - erf((z_lo - z_data[i])/(np.sqrt(2)*sigz[i])))
    
    #Replace the ra and dec of the galxies with the mid value of ra and dec pixel in which they are located
    for i in range(len(ra_data)):
        for j in range(len(ra_bin_edges)-1):
            if  ra_bin_edges[j] <= ra_data[i] < ra_bin_edges[j+1]:
                ra_data[i]=(ra_bin_edges[j] + ra_bin_edges[j+1])*0.5 
                  
               
    for i in range(len(dec_data)):
        for j in range(len(dec_bin_edges)-1):
            if  dec_bin_edges[j]  >= dec_data[i] > dec_bin_edges[j+1]:
                dec_data[i]=(dec_bin_edges[j] + dec_bin_edges[j+1])*0.5 
                
    galaxy_prob_pix=np.c_[ra_data,dec_data,z_prob_pix]    

    ###########add all the rows with the same ra and dec
    d = {}
    for sub in galaxy_prob_pix:
        d.setdefault(tuple(sub[:2]),[]).append(sub[2:])
    
    gal_pix= np.array([k+tuple(map(sum, zip(*v))) for k,v in d.items()]) 
    z_mid=[(z_bin_edges[i]+z_bin_edges[i+1])/2 for i in range(len(z_bin_edges)-1)]
    
    gw=[]
    
    for i in range(len(gal_pix)):
        gw.append(np.c_[[gal_pix[:,:2][i]]*len(z_mid),z_mid])  #list of arrays (ra,dec,zmid)
    
    gr= np.reshape(np.ravel(gw),(len(gal_pix)*len(z_mid),3)) # reshape to make it into list of list

    #print gr

    hh= np.delete(gal_pix,0,1)  #delete ra column

    gg= np.delete(hh,0,1)   # delete dec column

    prbs=np.ravel(np.reshape(np.ravel(gg),(len(gr[:,0]),1)))  #take the prob rows and turn them into one long vector to match the number of pixels

    #print len(prbs), len(gr[:,0])

    prob_data=np.c_[gr[:,0],gr[:,1],gr[:,2],prbs]

    ra=prob_data[:,0]
    dec=prob_data[:,1]
    rdsft=prob_data[:,2]
    w=prob_data[:,3]
            

    file2 = open("/home/nil/LNmocks/Gauss_distribute/"+"vox_"+file +".dat", "w")
    for index in range(len(ra)):
        file2.write(str(ra[index]) + "   " + str(dec[index])  + "   " + str(rdsft[index])  + "   " + str(w[index]) +  "\n")
    file2.close()


In [ ]:
aaa=np.array([[1, 2, 3, 4,4],[5, 6, 7, 8,6],[1,2,5,6,3], [5,6,2,3,1]])

d = {}
for sub in aaa:
     d.setdefault(tuple(sub[:2]),[]).append(sub[2:])
    
gal_pix= np.array([k+tuple(map(sum, zip(*v))) for k,v in d.items()]) 

print gal_pix
